# Solve MaxCut Using Quantum Approximate Optimization Algorithm (QAOA)
<ul>
<li>Contributors: IBM Quantum team and Research Technology Education team
<li>Contact for questions and technical support: IBM.Research.JupyterLab@ibm.com
<li>Provenance: IBM Research
<li>Version: 1.0.6
<li>Release date: 2023-12-01
<li>Compute requirements: CPU: estimated 2 minutes for default settings, 30 minutes or more for large graphs on real backends
<li>Memory requirements: 16 GB
<li>Notebook set: Quantum Notebooks
</ul>

# Demo Controls

Please review the documentation in the next cell for important demo controls. Here is some additional guidance.

If you want the output of the notebook printed on a dual screen, set `dual_screen` to True in the the next cell. If you don't want a dual screen, set `dual_screen` to False.

If `dual_screen` is True, do the following:
- Select **Kernel/Restart Kernel and Clear All Outputs**.
- Run the next cell.
- Right click on the same cell near the line at the bottom of the cell and select **Create New View for Output**.
- You will see a new panel at the bottom of your screen labeled **Output View**. Drag the **Output View** panel to the top right side of the JupyterLab and it will cover the whole right side of your screen when you drop it.
- Since the screen is getting a bit crowded, you can temporarily hide the File Browser by toggling the File Browser icon on the top left of the JupyterLab.
- Always run the notebook using **Kernel/Restart Kernel and Run All Cells**.

If you want to use a real Quantum backend, you will need a Quantum token that supports Sessions. Sessions are used in the optimization (Step 3). Please check with your administrator on Sessions support. If you choose a simulator as your backend, you will not need to token with Sessions support, but your graph will be limited to a maximum of 20 nodes.

Here are some recommended control sets:

1. A fast demo of a small graph on a simulator:

- n = 7                         (any number of nodes <= 20)
- use_coupling_map = False      (generate random graph)
- edge_density = 0.5            (fairly dense)
- opt_iterations = 30           (number of optimization iterations)
- num_shots = 10000             (number of sampling shots)
- num_bitstrings = 500000       (max number of top bitstrings to evaluate for best maxcut value)
- show_bitstrings = False       (T: can be used to debug small graphs)
- specific_backend = True       (use specific backend)
- resource = "FakeTokyo"        (good simulator for our purposes)
- skip_opt = False              (run optimization)

2. A fast demo showing saved results for n=100 on ibm_brisbane

- n = 99                        (n=99 is a special flag for showing saved results of a specific 100-node graph)

3. Random medium sized graph on a real backend

- n = 22                        (number of nodes)
- use_coupling_map = False      (generate random graph)
- edge_density = 0.4            (fairly dense)
- opt_iterations = 30           (number of optimization iterations)
- num_shots = 10000             (number of sampling shots)
- num_bitstrings = 500000       (max number of top bitstrings to evaluate for best maxcut value)
- show_bitstrings = False       (avoid using this flag on large graphs as it will exceed printing limits)
- specific_backend = True       (use specific backend)
- resource = "ibm_brisbane"     (avoid backends that don't converge for optimization)
- skip_opt = False              (run optimization)

4. Large graph on a real backend based on coupling map

- n = 100                       (select any number of nodes <= 127>
- use_coupling_map = True       (base graph on physical coupling map so circuit is not too deep)
- edge_density = 0.02           (edge_density is ignored for coupling map option)
- opt_iterations = 30           (number of optimization iterations)
- num_shots = 100000            (number of sampling shots)
- num_bitstrings = 500000       (max number of top bitstrings to evaluate for best maxcut value)
- show_bitstrings = False       (avoid using this flag on large graphs as it will exceed printing limits)
- specific_backend = True       (use specific backend)
- resource = "ibm_brisbane"     (avoid backends that don't converge for optimization)
- skip_opt = False              (run optimization)

In [1]:
dual_screen = True            # T: send output to second screen for demos

n = 8                     # number of nodes
                              #  for n<=20 you can specify a simulator or a real backend
                              #  for n>20 you must specify a real backend
                              #  n=99 is a special flag for showing saved results of a 100-node graph
use_coupling_map = False      # T: base graph on physical coupling map so circuit is not too deep, F: try a random graph
                              #  a physical coupling map reflects how a real backend's qubits are actually connected
edge_density = 0.3            # fraction of all possible edges that appear in random graphs
                              #  for large graphs, use low density (<= 0.03) to avoid circuit depth issues 
opt_iterations = 30           # number of optimization iterations
num_shots = 10000             # number of sampling shots
num_bitstrings = 500000       # max number of top bitstrings to evaluate for best maxcut value
show_bitstrings = False       # T: for debugging small graphs
specific_backend = True       # T: use specific backend; F: select least busy backend
                              # least busy backend is risky, you may get a backend that doesn't converge for optimization
resource = "FakeTokyo"        # use this simulator if specific_backend = True
#resource = "ibm_brisbane"    # use this real backend if specific_backend = True
skip_opt = False              # T: for faster runs when you already ran optimization and have a good resultx
resultx = [4.375e+00, 3.534e-01]   # got this value for n=100 with coupling map graph on ibm_brisbane

import matplotlib.pyplot as plt
import matplotlib
from IPython.display import Image, display
import ipywidgets as widgets
from ipynb_pause import flow

H1 = "<h1 style='font-family:IBM Plex Sans;font-size:28px'>"
H2 = "<h1 style='font-family:IBM Plex Sans;font-size:24px'>"
Norm = "<h1 style='font-family:IBM Plex Sans;font-size:20px'>"
Ex = "<h1 style='font-family:IBM Plex Sans;font-size:20px;font-style:italic'>"

out = widgets.Output(layout={'border': '1px solid black'})
run=flow.display_mode(mode=dual_screen, output=out, color='darkblue')
if dual_screen:
    display(out)

# for carbon compliant plots
# %run '/opt/ibm/visualization/matplotlib/matplotlib_template_dark.ipynb'

Output(layout=Layout(border='1px solid black'))

# Table of Contents

* [Summary](#nm_summary)
* [Quantum Approximate Optimization Algorithm (QAOA)](#qaoa)
* [Qiskit](#nm_qiskit)
* [Build the Qiskit Pattern](#nm_pattern)
    * [Step 0. Setup](#nm_set)
    * [Step 1. Map problem to Quantum circuits and operators](#nm_map)
    * [Step 2. Optimize for target hardware](#nm_opt)
    * [Step 3. Execute on target hardware](#nm_execute)
    * [Step 4. Post-process results](#nm_processing)
* [Deploy the Qiskit Pattern to the Cloud](#nm_deploy)
* [Run the Qiskit Pattern as a Managed Service](#nm_run)
* [Conclusion](#nm_conclusion)
* [Learn More](#nm_learn)

<a id="nm_summary"></a>
# Summary

[MaxCut](https://en.wikipedia.org/wiki/Maximum_cut) is an NP-complete graph problem where the goal is to divide the vertices into two sets such that the first set has the largest number of edges leading to the second set. A simple MaxCut example is displayed in the next cell. 

MaxCut has applications in clustering, network science, and statistical physics. As a practical example, consider a system of many people who can interact and influence each other's buying decisions. The goal is to implement a marketing strategy where products are offered for free to some individuals in order to maximize revenues. Individuals are represented by vertices of a graph, and their interactions as edges between vertices. Influence strength can be modeled by weights assigned to the edges. MaxCut is used to identify the optimal subset of individuals that should get the free products.

This notebook demonstrates how to solve MaxCut for a variety of graph sizes using the **Quantum Approximate Optimization Algorithm (QAOA)**, a hybrid quantum-classical iterative method. This notebook is based on [a notebook that is available on the Qiskit website](https://learning.quantum.ibm.com/tutorial/quantum-approximate-optimization-algorithm).

NOTE: Running this notebook requires that you register on the **IBM Quantum Platform** in order to submit Quantum jobs to an IBM Quantum computer or simulator.  Registration instructions are provided in the next cell.

<a id="qaoa"></a>
# Quantum Approximate Optimization Algorithm (QAOA)

To give some context before mapping this problem to a quantum algorithm, you can better understand how the Max-Cut problem becomes a classical combinatorial optimization problem by first considering the minimization of a function $f(x)$

$$
\min_{x\in \{0, 1\}^n}f(x),
$$

where the input $x$ is a vector whose components correspond to each node of a graph.  Then, constrain each of these components to be either $0$ or $1$ (which represent being included or not included in the cut). This small-scale example case uses a graph with $n=5$ nodes.

You could write a function of a pair of nodes $i,j$ which indicates whether the corresponding edge $(i,j)$ is in the cut. For example, the function $x_i + x_j - 2 x_i x_j$ is 1 only if one of either $x_i$ or $x_j$ are 1 (which means that the edge is in the cut) and zero otherwise. The problem of maximizing the edges in the cut can be formulated as

$$
\max_{x\in \{0, 1\}^n} \sum_{(i,j)} x_i + x_j - 2 x_i x_j,
$$

which can be rewritten as a minimization of the form

$$
\min_{x\in \{0, 1\}^n} \sum_{(i,j)}  2 x_i x_j - x_i - x_j.
$$

The minimum of $f(x)$ in this case is when the number of edges traversed by the cut is maximal. As you can see, there is nothing relating to quantum computing yet. We need to reformulate this problem into something that a quantum computer can understand.

<a id="nm_qiskit"></a>
# Qiskit

[Qiskit](https://qiskit.org) is an open-source software development kit for working with Quantum computers at the level of circuits, pulses, and algorithms. It provides tools for creating and manipulating Quantum programs and running them on prototype Quantum devices on the IBM Quantum Platform or on simulators on a local computer. 

[Qiskit Runtime](https://www.ibm.com/quantum/qiskit-runtime) is IBM's Quantum computing service and programming model for building, optimizing, and executing workloads at scale using Qiskit. Qiskit Runtime introduces primitives to seamlessly perform foundational Quantum computing tasks with increased performance. Qiskit Runtime improves operational efficiency by offering a containerized execution environment. Qiskit Runtime demonstrates dramatic performance increases over previous approaches, offering up to a 120X speedup for some applications. Through controllable error mitigation and suppression, Qiskit Runtime enables users to manage the performance of hardware systems. With error mitigation, Qiskit Runtime demonstrates significant improvement in the quality of results.

## Creating a Qiskit pattern

Creating utility-scale quantum applications typically requires a variety of compute resources. A **Qiskit Pattern** outlines an intuitive, repeatable set of steps for implementing a Quantum computing workflow.

Steps in a Qiskit Pattern:
1. Map problem to Quantum circuits and operators
2. Optimize for target hardware
3. Execute on target hardware
4. Post-process results

Once we have built a Qiskit Pattern, we can use [Quantum Serverless](https://docs.quantum.ibm.com/run/quantum-serverless) to deploy it and submit it for managed execution. This can be done in three steps:
1. Build the Qiskit Pattern
2. Deploy to the cloud
3. Run the Qiskit Pattern on managed compute resources

## Instructions for accessing Qiskit in this notebook

Running this notebook requires that you register on the [IBM Quantum Platform](https://quantum-computing.ibm.com/) in order to submit Quantum jobs.  This only needs to be done when this notebook set is installed or updated to a new version. To do this:
1. Log into the [IBM Quantum Platform](https://quantum-computing.ibm.com/).
2. Copy the API token on the top right.
3. In the File Browser on the left side of this JupyterLab, navigate to *Lab files/quantum-notebooks*.
4. Right click on the `config.json` file and select **Open with->Editor**.
5. Insert the copied API token in the *token* field within the quotation marks.
6. Save the file by clicking on *File->Save JSON File* and exit the file. 

Whenever you update this notebook set to a new version, you will need to re-insert your API token in the `config.json` file. The `config.json` file also contains a *resource* field, which is preset to one of the compute resources (Quantum computers and simulators) available to you on the [IBM Quantum Platform](https://quantum-computing.ibm.com/) website. You can also modify this field.

When running this notebook, log into the [IBM Quantum Platform](https://quantum-computing.ibm.com/) in another browser window. Quantum jobs are queued on compute resources, which sometimes causes a delay in notebook execution. You can monitor such delays in the Dashboard of the IBM Quantum Platform. 

<a id="nm_pattern"></a>
# Build the Qiskit Pattern

We will implement our pattern using the four steps outlined above.

<a id="nm_set"></a>
# Step 0. Setup

Import some generic packages.

In [2]:
import rustworkx as rx
from rustworkx.visualization import mpl_draw as draw_graph
import numpy as np
import math
import json
import qiskit
import qiskit_ibm_runtime
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Session, EstimatorV2 as Estimator
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_ibm_runtime.fake_provider import FakeTokyo         # 20 qubits
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.circuit.library import QAOAAnsatz
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import minimize
from typing import Sequence
import time

# make sure API token has been provided

with open('config.json') as data_file:
    data = json.load(data_file)
token = data['token']
if token == '':
    if dual_screen:
        with out:
            out.clear_output()
            display(widgets.HTML(Norm+"Error: No API token provided in config.json"))
    raise SystemExit("No API token provided in config.json")
    
# make sure real backend is provided for n>20

if n>20 and n!=99 and specific_backend and resource[:4]=="Fake":
    if dual_screen:
        with out:
            out.clear_output()
            display(widgets.HTML(Norm+"Error: Must use real backend for n>20"))
    raise SystemExit("Must use real backend for n>20")
    
# make sure use_coupling_map is being used properly

if use_coupling_map and specific_backend==True and resource[:4]=="Fake":
    if dual_screen:
        with out:
            out.clear_output()
            display(widgets.HTML(Norm+"Error: use_coupling_map cannot be applied to simulator"))
    raise SystemExit("use_coupling_map cannot be applied to simulator")
    
# make sure edge_density is valid

if edge_density <= 0.0 or edge_density > 1.0:
    if dual_screen:
        with out:
            out.clear_output()
            display(widgets.HTML(Norm+"Error: invalid edge_density"))
    raise SystemExit("invalid edge_density")
    
# establish a backend

QiskitRuntimeService.save_account(channel="ibm_quantum", token=token, overwrite=True, set_as_default=True)
service = QiskitRuntimeService(channel='ibm_quantum')
# service = QiskitRuntimeService(channel="ibm_quantum",token=token)
if specific_backend:
    if resource[:4]=="Fake":
        backend = eval(resource+"()")
    else:
        backend = service.backend(resource)
else:
    backend = service.least_busy(min_num_qubits=127)
    resource = backend.name

# display opening screen

if dual_screen:
    with out:
        out.clear_output()
        display(widgets.HTML(H1+"Step 0: Setup"))
        display(widgets.HTML(Norm+"MaxCut is an NP-complete graph problem where the goal is to divide the nodes into two sets such that the number of edges between the sets is maximized.</li>"))
        display(widgets.HTML(Norm+"Example application: Consider a marketing strategy in which products are offered for free to some customers. Identify the free customers in order to maximize revenues.</li>"))
        display(widgets.HTML(Norm+"Example network:</li>"))
        img = plt.imread('./data/img/maxcut.png')
        plt.figure(figsize = (2,2))
        plt.imshow(img)
        plt.axis('off')
        plt.show()
        display(widgets.HTML(Ex+"https://en.wikipedia.org/wiki/Maximum_cut"))
        display(widgets.HTML(Norm+"Compute resource: "+resource))
else:
    print("MaxCut is an NP-complete graph problem where the goal is to divide the nodes into two sets such that the number of edges between the sets is maximized.")
    print("Example application: Consider a marketing strategy in which products are offered for free to some customers. Identify the free customers in order to maximize revenues.")
    print("MaxCut example (https://en.wikipedia.org/wiki/Maximum_cut):")
    display(Image(filename='./data/img/maxcut.png',width=200))

print(f"Compute resource: {resource}")
print(f"Qiskit version: {qiskit.version.get_version_info()}")
print(f"Qiskit Runtime version: {qiskit_ibm_runtime.version.get_version_info()}")

run.pause()

<string>:1: DeprecationWarning: The class ``qiskit.providers.models.backendconfiguration.GateConfig`` is deprecated as of qiskit 1.2. It will be removed in the 2.0 release. The models in ``qiskit.providers.models`` are part of the deprecated `BackendV1` workflow  and no longer necessary for `BackendV2`. If a user workflow requires these representations it likely relies on deprecated functionality and should be updated to use `BackendV2`.
<string>:1: DeprecationWarning: The class ``qiskit.providers.models.backendconfiguration.QasmBackendConfiguration`` is deprecated as of qiskit 1.2. It will be removed in the 2.0 release. The models in ``qiskit.providers.models`` are part of the deprecated `BackendV1` workflow and no longer necessary for `BackendV2`. If a user workflow requires these representations it likely relies on deprecated functionality and should be updated to use `BackendV2`.


Compute resource: FakeTokyo
Qiskit version: 1.2.4
Qiskit Runtime version: 0.24.0


Button(description='Continue', style=ButtonStyle(button_color='darkblue'))

<a id="nm_map"></a>
# Step 1: Map problem to Quantum circuits and operators

## Create a classical graph

We provide options for small and large (utility scale) graphs.

In [3]:
def random_graph(n, density):
    """ Build a random graph of a given edge density
    If N is 5, 10, or 100, predefined graphs are created where we know the MaxCut values.
    Otherwise, random graphs are returned.

    Parameters:
        N (int): Number of nodes
        density (float): Fraction of all possible edges that appear in the graph (between 0.0 and 1.0)

    Returns:
        ndarray: Adjacency matrix as NumPy array
    """
    global resource
    if n==99:      # special case where saved results of 100 node graph will be shown
        N = 100
    else:
        N = n      # actual number of nodes in graph, overriding 99 flag
    g = rx.PyGraph()
    g.add_nodes_from(np.arange(0, N, 1))
    max_edges = N*(N-1)//2
    if n==99:   # special case where saved results of a specific 100 node graph will be shown
        edge_list = [(0,1,1.0), (0,14,1.0),
                     (1,2,1.0),
                     (2,3,1.0),
                     (3,4,1.0),
                     (4,5,1.0), (4,15,1.0),
                     (5,6,1.0),
                     (6,7,1.0),
                     (7,8,1.0),
                     (8,9,1.0), (8,16,1.0),
                     (9,10,1.0),
                     (10,11,1.0),
                     (11,12,1.0),
                     (12,13,1.0), (12,17,1.0),
                     (14,18,1.0),
                     (15,22,1.0),
                     (16,26,1.0),
                     (17,30,1.0),
                     (18,19,1.0),
                     (19,20,1.0),
                     (20,21,1.0), (20,33,1.0),
                     (21,22,1.0),
                     (22,23,1.0),
                     (23,24,1.0),
                     (24,25,1.0), (24,34,1.0),
                     (25,26,1.0),
                     (26,27,1.0),
                     (27,28,1.0),
                     (28,29,1.0), (28,35,1.0),
                     (29,30,1.0),
                     (30,31,1.0),
                     (31,32,1.0),
                     (32,36,1.0),
                     (33,39,1.0),
                     (34,43,1.0),
                     (35,47,1.0),
                     (36,51,1.0),
                     (37,38,1.0), (37,52,1.0),
                     (38,39,1.0),
                     (39,40,1.0),
                     (40,41,1.0),
                     (41,42,1.0), (41,53,1.0),
                     (42,43,1.0),
                     (43,44,1.0),
                     (44,45,1.0),
                     (45,46,1.0), (45,54,1.0),
                     (46,47,1.0),
                     (47,48,1.0),
                     (48,49,1.0),
                     (49,50,1.0), (49,55,1.0),
                     (50,51,1.0),
                     (52,56,1.0),
                     (53,60,1.0),
                     (54,64,1.0),
                     (55,68,1.0),
                     (56,57,1.0),
                     (57,58,1.0),
                     (58,59,1.0), (58,71,1.0),
                     (59,60,1.0),
                     (60,61,1.0),
                     (61,62,1.0),
                     (62,63,1.0), (62,72,1.0),
                     (63,64,1.0),
                     (64,65,1.0),
                     (65,66,1.0),
                     (66,67,1.0), (66,73,1.0),
                     (67,68,1.0),
                     (68,69,1.0),
                     (69,70,1.0),
                     (70,74,1.0),
                     (71,77,1.0),
                     (72,81,1.0),
                     (73,85,1.0),
                     (74,89,1.0),
                     (75,76,1.0), (75,90,1.0),
                     (76,77,1.0),
                     (77,78,1.0),
                     (78,79,1.0),
                     (79,80,1.0), (79,91,1.0),
                     (80,81,1.0),
                     (81,82,1.0),
                     (82,83,1.0),
                     (83,84,1.0), (83,92,1.0),
                     (84,85,1.0),
                     (85,86,1.0),
                     (86,87,1.0),
                     (87,88,1.0), (87,93,1.0),
                     (88,89,1.0),
                     (90,94,1.0),
                     (91,98,1.0),
                     (94,95,1.0),
                     (95,96,1.0),
                     (96,97,1.0),
                     (97,98,1.0),
                     (98,99,1.0)]
        E = len(edge_list)
        density = E/max_edges
        print(f"Creating predefined graph where n={N}, e={E}, density={density}, MaxCut value = 84")
    elif use_coupling_map:    # sparse graph that runs faster
        edge_list = []
        for edge in backend.coupling_map:
            if edge[0] < N and edge[1] < N:
                edge_list.append((edge[0], edge[1], 1.0))
        E = len(edge_list)
        density = E/max_edges
        print(f"Creating graph based on coupling map where n={N}, e={E}, density={density}, MaxCut value = unknown")
    else:         # here n<=20 and not 5 or 10 where graph is predefined
        edge_list = []
        E = math.floor(max_edges * density)
        print(f"Creating random graph where n={N}, e={E}, density={density}, MaxCut value = unknown")
        inds = np.sort(np.random.choice(max_edges, size=E, replace=False))
        for k in inds:
            i = N - 2 - int(math.sqrt(-8*k + 4*N*(N-1)-7)/2 - 0.5)
            j = (k + i + 1 - N*(N-1)//2 + (N-i)*((N-i)-1)//2)
            edge_list.append((i, j, 1.0))
            print(f"Creating edge ({i},{j})")
    g.add_edges_from(edge_list)
    return g

graph = random_graph(n, edge_density)
if n==99:
    N = 100
else:
    N = n
E = len(graph.edge_list())
max_edges = N*(N-1)//2
density = E/max_edges

# Print graph

node_size = 300
if dual_screen:
    with out:
        out.clear_output()
        display(widgets.HTML(H1+"Step 1. Map problem to Quantum circuits and operators"))
        display(widgets.HTML(Norm + "Input graph with " + str(N) + " nodes and " + str(E) + " edges (density " + str(round(density,2)) +"):"))
        draw_graph(graph, node_color='cyan', node_size=node_size, with_labels=True, width=1)
        plt.show()
else:
    print("Input graph:")
    draw_graph(graph, node_color='cyan', node_size=node_size, with_labels=True, width=1)

Creating random graph where n=8, e=8, density=0.3, MaxCut value = unknown
Creating edge (0,4)
Creating edge (1,3)
Creating edge (1,4)
Creating edge (2,4)
Creating edge (3,6)
Creating edge (4,6)
Creating edge (5,6)
Creating edge (6,7)


## Map the graph to a Quantum problem

Map the classical problem (graph) into quantum **circuits** and **operators**. To do this, there are three main steps to take:

1. Utilize a series of mathematical reformulations, to represent this problem using the Quadratic Unconstrained Binary Optimization (QUBO) problems notation.
2. Rewrite the optimization problem as a Hamiltonian for which the ground state corresponds to the solution which minimizes the cost function.
3. Create a quantum circuit which will prepare the ground state of this Hamiltonian via a process similar to quantum annealing.

**Note:** In the QAOA methodology, we ultimately want to have an operator (**Hamiltonian**) that represents the **cost function** of our hybrid algorithm, as well as a parametrized circuit (**Ansatz**) that represents quantum states with candidate solutions to the problem. You can sample from these candidate states and then evaluate them using the cost function.

### Graph → optimization problem

The first step of the mapping is a notation change, The following expresses the problem in QUBO notation:

$$
\min_{x\in \{0, 1\}^n}x^T Q x,
$$

where $Q$ is a $n\times n$ matrix of real numbers, $n$ corresponds to the number of nodes in your graph, $x$ is the vector of binary variables introduced above, and $x^T$ indicates the transpose of the vector $x$.

```
Maximize
 -2*x_0*x_1 - 2*x_0*x_2 - 2*x_0*x_4 - 2*x_1*x_2 - 2*x_2*x_3 - 2*x_3*x_4 + 3*x_0
 + 2*x_1 + 3*x_2 + 2*x_3 + 2*x_4

Subject to
  No constraints

  Binary variables (5)
    x_0 x_1 x_2 x_3 x_4
```

### Optimization problem → Hamiltonian

You can then reformulate the QUBO problem as a **Hamiltonian** (here, a matrix that represents the energy of a system):

$$
H_C=\sum_{ij}Q_{ij}Z_iZ_j + \sum_i b_iZ_i.
$$

> **Reformulation steps from the QAOA problem to the Hamiltonian**

> To demonstrate how the QAOA problem can be rewritten in this way, first replace the binary variables $x_i$ to a new set of variables $z_i\in\{-1, 1\}$ via
>
> $$
> x_i = \frac{1-z_i}{2}.
> $$
>
> Here you can see that if $x_i$ is $0$, then $z_i$ must be $1$. When the $x_i$'s are substituted for the $z_i$'s in the optimization problem ($x^TQx$), an equivalent formulation can be obtained.
>
> $$
> x^TQx=\sum_{ij}Q_{ij}x_ix_j \\ =\frac{1}{4}\sum_{ij}Q_{ij}(1-z_i)(1-z_j) \\=\frac{1}{4}\sum_{ij}Q_{ij}z_iz_j-\frac{1}{4}\sum_{ij}(Q_{ij}+Q_{ji})z_i + \frac{n^2}{4}.
> $$
>
> Now if we define $b_i=-\sum_{j}(Q_{ij}+Q_{ji})$, remove the prefactor, and the constant $n^2$ term, we arrive at the two equivalent formulations of the same optimization problem.
>
> $$
> \min_{x\in\{0,1\}^n} x^TQx\Longleftrightarrow \min_{z\in\{-1,1\}^n}z^TQz + b^Tz
> $$
>
> Here, $b$ depends on $Q$. Note that to obtain $z^TQz + b^Tz$ we dropped the factor of 1/4 and a constant offset of $n^2$ which do not play a role in the optimization.
>
> Now, to obtain a quantum formulation of the problem, promote the $z_i$ variables to a Pauli $Z$ matrix, such as a $2\times 2$ matrix of the form
>
> $$
> Z_i = \begin{pmatrix}1 & 0 \\ 0 & -1\end{pmatrix}.
> $$
>
> When you substitute these matrices in the optimization problem above, you obtain the following Hamiltonian
>
> $$
> H_C=\sum_{ij}Q_{ij}Z_iZ_j + \sum_i b_iZ_i.
> $$
>
> *Also recall that the $Z$ matrices are embedded in the quantum computer's computational space, i.e., a Hilbert space of size $2^n\times 2^n$. Therefore, you should understand terms such as $Z_iZ_j$ as the tensor product $Z_i\otimes Z_j$ embedded in the $2^n\times 2^n$ Hilbert space. For example, in a problem with five decision variables the term $Z_1Z_3$ is understood to mean $I\otimes Z_3\otimes I\otimes Z_1\otimes I$ where $I$ is the $2\times 2$ identity matrix.*

This Hamiltonian is called the <b>cost function Hamiltonian</b>. It has the property that its ground state corresponds to the solution that <b>minimizes the cost function $f(x)$</b>.
Therefore, to solve your optimization problem you now need to prepare the ground state of $H_C$ (or a state with a high overlap with it) on the quantum computer. Then, sampling from this state will, with a high probability, yield the solution to $min~f(x)$.

In [4]:
def build_max_cut_paulis(graph: rx.PyGraph) -> list[tuple[str, float]]:
    """Convert the graph to Pauli list.
    This function does the inverse of `build_max_cut_graph`
    """
    pauli_list = []
    for edge in list(graph.edge_list()):
        paulis = ["I"] * len(graph)
        paulis[edge[0]], paulis[edge[1]] = "Z", "Z"
        weight = graph.get_edge_data(edge[0], edge[1])
        pauli_list.append(("".join(paulis)[::-1], weight))
    return pauli_list

max_cut_paulis = build_max_cut_paulis(graph)
cost_hamiltonian = SparsePauliOp.from_list(max_cut_paulis)
print("Cost Function Hamiltonian:", cost_hamiltonian)

Cost Function Hamiltonian: SparsePauliOp(['IIIZIIIZ', 'IIIIZIZI', 'IIIZIIZI', 'IIIZIZII', 'IZIIZIII', 'IZIZIIII', 'IZZIIIII', 'ZZIIIIII'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j])


### Hamiltonian → quantum circuit

The Hamiltonian $H_c$ contains the quantum definition of our problem. Now we can create a quantum circuit that will help *sample* good solutions from the quantum computer. The QAOA is inspired by quantum annealing and applies alternating layers of operators in the quantum circuit.

The general idea is to start in the ground state of a known system, $H^{\otimes n}|0\rangle$ above, and then steer the system into the ground state of the cost operator that you are interested in. This is done by applying the operators $\exp\{-i\gamma_k H_C\}$ and $\exp\{-i\beta_k H_m\}$ with angles $\gamma_1,...,\gamma_p$ and $\beta_1,...,\beta_p~$.

The quantum circuit that we generate is **parametrized** by $\gamma_i$ and $\beta_i$, so you can try out different values of $\gamma_i$ and $\beta_i$ and sample from the resulting state.

![Circuit diagram with QAOA layers](https://learning-api.quantum.ibm.com/assets/29a70f21-b453-4df7-b726-19468e5b1f51)

In this case, we will try an example with one QAOA layer that contains two parameters: $\gamma_1$ and $\beta_1$.

In [5]:
if n>20:
    reps = 1
else:
    reps = 2
circuit = QAOAAnsatz(cost_operator=cost_hamiltonian, reps=reps)
circuit.measure_all()

if dual_screen:
    with out:
        display(widgets.HTML(Norm+"Abstract QAOA Ansatz circuit:"))
        display(circuit.draw('mpl', fold=False, idle_wires=False))
else:
    print("Abstract QAOA Ansatz circuit:")
    display(circuit.draw('mpl', fold=False, idle_wires=False))

run.pause()

Button(description='Continue', style=ButtonStyle(button_color='darkblue'))

In [6]:
circuit.parameters

ParameterView([ParameterVectorElement(β[0]), ParameterVectorElement(β[1]), ParameterVectorElement(γ[0]), ParameterVectorElement(γ[1])])

<a id="nm_opt"></a>
# Step 2: Optimize for target hardware

The circuit above contains a series of abstractions useful to think about quantum algorithms, but not possible to run on the hardware. To be able to run on a QPU, the circuit needs to undergo a series of operations that make up the **transpilation** or **circuit optimization** step of the pattern.

The Qiskit library offers a series of **transpilation passes** that cater to a wide range of circuit transformations. You need to make sure that your circuit is **optimized** for your purpose.

Transpilation may involves several steps, such as:

* **Initial mapping** of the qubits in the circuit (such as decision variables) to physical qubits on the device.
* **Unrolling** of the instructions in the quantum circuit to the hardware-native instructions that the backend understands.
* **Routing** of any qubits in the circuit that interact to physical qubits that are adjacent with one another.
* **Error suppression** by adding single-qubit gates to suppress noise with dynamical decoupling.

More information about transpilation is available in our [documentation](https://docs.quantum.ibm.com/transpile).

The following code transforms and optimizes the abstract circuit into a format that is ready for execution on one of devices accessible through the cloud using the **Qiskit IBM Runtime service**.

In [7]:
# for n=99 show saved candidate circuit

if n==99:
    if dual_screen:
        with out:
            display(widgets.HTML(H1+"Step 2. Optimize for target hardware"))
            display(widgets.HTML(Norm+"Candidate circuit:"))
            display(Image(filename='./data/img/100_candidate_circuit.png'))
    else:
        print("Candidate circuit:")
        display(Image(filename='./data/img/100_candidate_circuit.png'))

# build candidate circuit
        
else:
    pm = generate_preset_pass_manager(optimization_level=3, backend=backend)
    candidate_circuit = pm.run(circuit)
    if n>20:
        scale = 0.1
    else:
        scale = 0.2
    
    if dual_screen:
        with out:
            display(widgets.HTML(H1+"Step 2. Optimize for target hardware"))
            display(widgets.HTML(Norm+"Candidate circuit:"))
            display(candidate_circuit.draw('mpl', scale=scale, fold=False, idle_wires=False))
            display(widgets.HTML(Norm+"Optimizing the circuit -- may take a few minutes."))
    else:
        print("Candidate circuit:")
        display(candidate_circuit.draw('mpl', scale=scale, fold=False, idle_wires=False))
        print("Optmizing the circuit -- may take a few minutes.")

    # record start time

    st = time.time()

/tmp/ipykernel_1604/1395732816.py:16: DeprecationWarning: The `generate_preset_pass_manager` function will stop supporting inputs of type `BackendV1` ( fake_tokyo ) in the `backend` parameter in a future release no earlier than 2.0. `BackendV1` is deprecated and implementations should move to `BackendV2`.
  pm = generate_preset_pass_manager(optimization_level=3, backend=backend)


<a id="nm_execute"></a>
# Step 3: Execute on target hardware

In the QAOA workflow, the optimal QAOA parameters are found in an iterative optimization loop, which runs a series of circuit evaluations and uses a classical optimizer to find the optimal $\beta_k$ and $\gamma_k$ parameters. This execution loop is executed via the following steps:

1. Define the initial parameters
2. Instantiate a new `Session` containing the optimization loop and the primitive used to sample the circuit
3. Once an optimal set of parameters is found, execute the circuit a final time to obtain a final distribution which will be used in the post-process step.

### Define circuit with initial parameters

We start with arbitrary chosen parameters.

### Backend and execution primitives

Use the **Qiskit Runtime primitives** to interact with IBM® backends. The two primitives are Sampler and Estimator, and the choice of primitive depends on what type of measurement you want to run on the quantum computer. For the minimization of $H_c$, use the Estimator since the measurement of the cost function is simply the expectation value of $\langle H_c \rangle$.

### Run

The primitives offer a variety of [execution modes](https://docs.quantum.ibm.com/run/execution-modes) to schedule workloads on quantum devices, and a QAOA workflow runs iteratively in a session.

![Illustration showing the behavior of Single job, Batch, and Session runtime modes.](https://learning-api.quantum.ibm.com/assets/73e400b1-e5a9-4ee8-9842-3e2bbfbef3f5)

We will plug the sampler-based cost function into the SciPy minimizing routine to find the optimal parameters.

In [8]:
initial_gamma = np.pi
initial_beta = np.pi/2
iteration = 0
if reps==1:
    init_params = [initial_gamma, initial_beta]
else:  # assume reps==2
    init_params = [initial_gamma, initial_beta, initial_gamma, initial_beta]
objective_func_vals = []  # global used in cost_func_estimator
    
def cost_func_estimator(params, ansatz, hamiltonian, estimator):
    # transform the observable defined on virtual qubits to
    # an observable defined on all physical qubits
    isa_hamiltonian = hamiltonian.apply_layout(ansatz.layout)
    pub = (ansatz, isa_hamiltonian, params)
    job = estimator.run([pub])
    results = job.result()[0]
    cost = results.data.evs
    global iteration
    iteration += 1
    print(f"Iteration {iteration} cost = {cost}")
    objective_func_vals.append(cost)
    return cost

# calculate optimal QAOA parameters

if skip_opt:
    print(f"Skipping optimization, resultx = {resultx}")
elif n!=99:
    with Session(backend=backend) as session:
        estimator = Estimator(mode=session)
        estimator.options.default_shots = 1000
        # Set simple error suppression/mitigation options
        estimator.options.dynamical_decoupling.enable = True
        estimator.options.dynamical_decoupling.sequence_type = "XY4"
    #    estimator.options.twirling.enable_gates = True                  -- commented out to speed up
    #    estimator.options.twirling.num_randomizations = "auto"          -- commented out to speed up
        result = minimize(
            cost_func_estimator,
            init_params,
            args=(candidate_circuit, cost_hamiltonian, estimator),
            method="COBYLA",
            options={'maxiter':opt_iterations}
        )
    resultx = result.x
    print("Optimization result:")
    print(result)

/opt/ibm/conda/envs/quantum-notebooks-1.0.0/lib/python3.11/site-packages/qiskit_ibm_runtime/fake_provider/local_service.py:247: UserWarning: Options {'default_shots': 1000, 'dynamical_decoupling': {'enable': True, 'sequence_type': 'XY4'}} have no effect in local testing mode.
  warnings.warn(f"Options {options_copy} have no effect in local testing mode.")


Iteration 1 cost = 0.115234375


/opt/ibm/conda/envs/quantum-notebooks-1.0.0/lib/python3.11/site-packages/qiskit_ibm_runtime/fake_provider/local_service.py:247: UserWarning: Options {'default_shots': 1000, 'dynamical_decoupling': {'enable': True, 'sequence_type': 'XY4'}} have no effect in local testing mode.
  warnings.warn(f"Options {options_copy} have no effect in local testing mode.")


Iteration 2 cost = 0.03369140625


/opt/ibm/conda/envs/quantum-notebooks-1.0.0/lib/python3.11/site-packages/qiskit_ibm_runtime/fake_provider/local_service.py:247: UserWarning: Options {'default_shots': 1000, 'dynamical_decoupling': {'enable': True, 'sequence_type': 'XY4'}} have no effect in local testing mode.
  warnings.warn(f"Options {options_copy} have no effect in local testing mode.")


...

Iteration 28 cost = -0.9697265625


/opt/ibm/conda/envs/quantum-notebooks-1.0.0/lib/python3.11/site-packages/qiskit_ibm_runtime/fake_provider/local_service.py:247: UserWarning: Options {'default_shots': 1000, 'dynamical_decoupling': {'enable': True, 'sequence_type': 'XY4'}} have no effect in local testing mode.
  warnings.warn(f"Options {options_copy} have no effect in local testing mode.")


Iteration 29 cost = -1.0859375


/opt/ibm/conda/envs/quantum-notebooks-1.0.0/lib/python3.11/site-packages/qiskit_ibm_runtime/fake_provider/local_service.py:247: UserWarning: Options {'default_shots': 1000, 'dynamical_decoupling': {'enable': True, 'sequence_type': 'XY4'}} have no effect in local testing mode.
  warnings.warn(f"Options {options_copy} have no effect in local testing mode.")


Iteration 30 cost = -0.939453125
Optimization result:
 message: Maximum number of function evaluations has been exceeded.
 success: False
  status: 2
     fun: -1.10498046875
       x: [ 3.346e+00  2.566e+00  4.039e+00  2.929e+00]
    nfev: 30
   maxcv: 0.0


In [9]:
# for n=99 show saved optimization results

if n==99:
    if dual_screen:
        with out:
            display(widgets.HTML(Norm+"Optimization results:"))
            display(Image(filename='./data/img/100_opt_results.png'))
    else:
        print("Optimization results:")
        display(Image(filename='./data/img/100_opt_results.png'))

# show optimization time and results
        
elif not skip_opt:
    et = time.time()
    print(f"Optimization time: {round((et-st)/60,2)} minutes")
    if dual_screen:
        with out:
            display(widgets.HTML(Norm+"Optimization time: "+str(round((et-st)/60,2))+" minutes"))
            display(widgets.HTML(Norm+"Optimization results:"))
            plt.figure(figsize=(12, 6))
            plt.plot(objective_func_vals)
            plt.xlabel("Iteration")
            plt.ylabel("Cost")
            plt.show()
    else:
        print("Optimization results:")
        plt.figure(figsize=(12, 6))
        plt.plot(objective_func_vals)
        plt.xlabel("Iteration")
        plt.ylabel("Cost")
        plt.show()

Optimization time: 0.97 minutes


Once we have found the optimal parameters for the circuit, we assign these parameters and sample the final distribution obtained with the optimized parameters. Here is where the *Sampler* primitive should be used since it is the probability distribution of bitstring measurements which correspond to the optimal cut of the graph.

**Note:** This means preparing a quantum state $\psi$ in the computer and then measuring it. A measurement will collapse the state into a single computational basis state - for example, `010101110000...` - which corresponds to a candidate solution $x$ to our initial optimization problem ($\max f(x)$ or $\min f(x)$ depending on the task).

In [10]:
run.pause()

# for n=99 show saved optimal circuit

if n==99:
    if dual_screen:
        with out:
            display(widgets.HTML(H1+"Step 3. Execute on target hardware"))
            display(widgets.HTML(Norm+"Optimized circuit:"))
            display(Image(filename='./data/img/100_opt_circuit.png'))
            display(widgets.HTML(Norm+"Sampling the circuit"))
    else:
        print("Optimized circuit:")
        display(Image(filename='./data/img/100_opt_circuit.png'))
        
else:
    
    # create optimized circuit

    optimized_circuit = candidate_circuit.assign_parameters(resultx)
    
    if n>20:
        scale = 0.1
    else:
        scale = 0.2
    if dual_screen:
        with out:
            out.clear_output()
            display(widgets.HTML(H1+"Step 3. Execute on target hardware"))
            display(widgets.HTML(Norm+"Optimized circuit:"))
            display(optimized_circuit.draw('mpl', scale=scale, fold=False, idle_wires=False))
            display(widgets.HTML(Norm+"Sampling the circuit -- may take a few minutes."))
    else:
        print("Optimized circuit:")
        display(optimized_circuit.draw('mpl', scale=scale, fold=False, idle_wires=False))
        print("Sampling the circuit -- may take a few minutes.")
    
    # record the start time

    st = time.time()

Button(description='Continue', style=ButtonStyle(button_color='darkblue'))

In [11]:
if n!=99:
    sampler = Sampler(mode=backend)
    sampler.options.default_shots = num_shots

    # Set simple error suppression/mitigation options
    
    sampler.options.dynamical_decoupling.enable = True
    sampler.options.dynamical_decoupling.sequence_type = "XY4"
    sampler.options.twirling.enable_gates = True
    sampler.options.twirling.num_randomizations = "auto"

    pub = (optimized_circuit, )
    job = sampler.run([pub], shots=num_shots)
    counts_int = job.result()[0].data.meas.get_int_counts()
    counts_bin = job.result()[0].data.meas.get_counts()
    shots = sum(counts_int.values())
    final_distribution_int = {key: val/shots for key, val in counts_int.items()}
    final_distribution_bin = {key: val/shots for key, val in counts_bin.items()}
    
    # display sampling time
    
    et = time.time()
    if dual_screen:
        with out:
            display(widgets.HTML(Norm+"Sampling time: "+str(round((et-st)/60,2))+" minutes"))
    print(f"Sampling time: {round((et-st)/60,2)} minutes")

/opt/ibm/conda/envs/quantum-notebooks-1.0.0/lib/python3.11/site-packages/qiskit_ibm_runtime/fake_provider/local_service.py:247: UserWarning: Options {'dynamical_decoupling': {'enable': True, 'sequence_type': 'XY4'}, 'twirling': {'enable_gates': True, 'num_randomizations': 'auto'}} have no effect in local testing mode.
  warnings.warn(f"Options {options_copy} have no effect in local testing mode.")


Sampling time: 0.04 minutes


<a id="nm_processing"></a>
# Step 4: Post-process results

The post-processing step analyzes the sampling output to return a solution for our original problem. The sampling output is a set of 2^^n bitstrings. This can be a very large number of bitstrings if n is large. We want to find the bitstring with the largest MaxCut value (number of edges connecting the two subsets). The bitstring with the highest probability is not necessarily the bitstring with the largest MaxCut value. We have to find the top bitstrings and evaluate their MaxCut values.

In [12]:
run.pause()

def evaluate_sample(x: Sequence[int], graph: rx.PyGraph) -> float:
    assert len(x) == len(list(graph.nodes())), "The length of x must coincide with the number of nodes in the graph."
    return sum(x[u] * (1 - x[v]) + x[v] * (1 - x[u]) for u, v in list(graph.edge_list()))

def visualize_counts(probs, num_qubits):
    """Visualize the outputs from the Qiskit Sampler primitive."""
    top = dict(sorted(probs.items(), key=lambda item: item[1], reverse=True)[:40])
    xvals, yvals = list(zip(*top.items()))
    plt.bar(xvals, yvals, color="tab:blue")
    plt.xticks(rotation=75)
    plt.title("Top measured bitstrings")
    plt.xlabel("Measured bitstring")
    plt.ylabel("Probability")
    plt.show()
    return top

# for n=99 show saved top bitstrings

if n==99:
    if dual_screen:
        with out:
            out.clear_output()
            display(widgets.HTML(H1+"Step 4. Post-process results"))
            display(Image(filename='./data/img/100_bitstrings.png'))
    else:
        display(Image(filename='./data/img/100_bitstrings.png'))
        
else:
    
    # display some of the top bitstrings
    
    if dual_screen:
        with out:
            out.clear_output()
            display(widgets.HTML(H1+"Step 4. Post-process results"))
            top = visualize_counts(final_distribution_bin, len(graph))
    else:
        top = visualize_counts(final_distribution_bin, len(graph))
        
    # obtain a larger set of top bitstrings
    
    top = dict(sorted(final_distribution_bin.items(), key=lambda item: item[1], reverse=True)[:num_bitstrings])

Button(description='Continue', style=ButtonStyle(button_color='darkblue'))

In [13]:
# find the bitstring from the top bitstrings that has the highest MaxCut value

if n!=99:
    cut_value = 0.0
    best_sol_bitstring = ''
    best_int_list = []
    i = 0
    for bit_str, prob in top.items():
        string_list = list(bit_str)
        int_list = []
        for s in string_list:
            int_list.append(int(s))
        int_list.reverse()
        cut_value_new = evaluate_sample(int_list, graph)
        i+=1
        if show_bitstrings:
            print(f"Evaluating {i}: {bit_str} --> {cut_value_new}")
        if cut_value_new > cut_value:
            cut_value = cut_value_new
            best_sol_bitstring = bit_str
            best_int_list = int_list

    if dual_screen:
        with out:
            display(widgets.HTML(Norm+"MaxCut bitstring: " + best_sol_bitstring))
            display(widgets.HTML(Norm+"The value of the cut is: " + str(cut_value)))

    print("MaxCut bitstring: ", best_sol_bitstring)
    print('The value of the cut is:', cut_value)
    
run.pause()

MaxCut bitstring:  10111000
The value of the cut is: 8


Button(description='Continue', style=ButtonStyle(button_color='darkblue'))

Visualize the best cut in the original graph

In [ ]:
# for n=99 show saved best graph

if n==99:
    if dual_screen:
        with out:
            display(Image(filename='./data/img/100_best_graph.png'))
    else:
        display(Image(filename='./data/img/100_best_graph.png'))
        
else:
    colors = ["tab:pink" if i==0 else "tab:cyan" for i in best_int_list]
    node_size = 300
    if dual_screen:
        with out:
            out.clear_output()
            display(widgets.HTML(H1+"Step 4. Post-process results"))
            draw_graph(graph, node_color=colors, node_size=node_size, with_labels=True, width=1)
            plt.show()
            display(widgets.HTML(Norm+"The value of the cut is: " + str(cut_value)))
    else:
        draw_graph(graph, node_color=colors, node_size=node_size, with_labels=True, width=1)
        print('The value of the cut is:', cut_value)
    
run.resume()

<a id="nm_conclusion"></a>
# Conclusion

This notebook presented Python code that solves a sample MaxCut problem using Qiskit on an IBM Quantum computer or simulator. MaxCut has applications in clustering, network science, and statistical physics. As a practical example, this notebook focused on a system of many people who can interact and influence each other's buying decisions. MaxCut was used to identify the optimal subset of individuals that should get the free products.

Users can create their own MaxCut solutions by uploading their data and customizing this code.

<a id="nm_learn"></a>
# Learn More

- [IBM Quantum Learning](https://learning.quantum-computing.ibm.com/) provides an extensive portfolio of educational content on Quantum computing.
- [IBM Quantum Accelerator](https://research.ibm.com/blog/quantum-accelerator-program) is a full-service offering for clients looking to build on access offered by our Premium Plan. Quantum Accelerator clients get personalized support plans to fully prepare for the coming impact of Quantum in their industries.
- [IBM Quantum GitHub](https://github.com/jaygambetta/my-notebooks) is an open source code repository for IBM Quantum examples.

If you have any questions or wish to schedule a technical deep dive, contact us by email at IBM.Research.JupyterLab@ibm.com.

© 2024 IBM Corporation